<a href="https://colab.research.google.com/github/Srajan1122/Hyperspectral_Image_Classification/blob/srajan/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [112]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Importing Libraries

In [0]:
import pandas as pd
import numpy as np
from scipy import io
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder 
from sklearn import preprocessing

# Importing Dataset

In [0]:
loaded_dataset = io.loadmat('/content/drive/My Drive/Srajan/Internship/SEM4-LeadingindiaAI/Tasks/Datasets/PaviaC/Pavia.mat')
for key, value in loaded_dataset.items():
  if isinstance(value, type(np.array([1]))):
    image = loaded_dataset[key]

In [0]:
ground_truth_1 = io.loadmat('/content/drive/My Drive/Srajan/Internship/SEM4-LeadingindiaAI/Tasks/Datasets/PaviaC/Pavia_gt.mat')
for key, value in ground_truth_1.items():
  if isinstance(value, type(np.array([1]))):
    ground_truth = ground_truth_1[key]

## Class Labels

In [0]:
label_values = ["Alfalfa", "Corn-notill", "Corn-mintill",
                        "Corn", "Grass-pasture", "Grass-trees",
                        "Grass-pasture-mowed", "Hay-windrowed", "Oats",
                        "Soybean-notill", "Soybean-mintill", "Soybean-clean",
                        "Wheat", "Woods", "Buildings-Grass-Trees-Drives",
                        "Stone-Steel-Towers"]

## Resizing the data

In [0]:
image_with_gt = np.dstack((image, ground_truth))

In [0]:
final_output = image_with_gt.reshape(ground_truth.size, image.shape[2]+1)

## Data Visualization in pandas

In [0]:
data = pd.DataFrame(final_output)

In [120]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102
0,854,601,350,266,138,118,178,194,257,269,239,243,220,191,173,150,125,131,137,172,195,218,255,291,320,346,347,368,404,428,429,416,394,368,357,361,369,368,351,328,...,219,239,289,366,482,632,814,1005,1225,1464,1732,2044,2383,2716,2985,3203,3385,3538,3667,3696,3548,3608,3744,3792,3802,3815,3836,3827,3765,3752,3759,3773,3779,3752,3690,3671,3664,3636,3643,0
1,527,642,575,294,123,168,207,154,209,299,299,278,235,181,181,236,256,253,225,202,221,256,321,358,385,429,492,533,554,553,557,580,561,522,525,513,474,435,412,401,...,254,305,349,401,508,681,888,1076,1297,1577,1855,2117,2456,2772,3002,3205,3388,3526,3629,3665,3586,3653,3747,3813,3831,3829,3841,3885,3916,3907,3873,3902,3921,3861,3854,3882,3834,3725,3768,0
2,374,322,179,87,169,268,360,339,286,309,340,305,206,202,269,256,246,226,218,275,318,356,395,425,461,520,587,623,621,617,636,657,665,670,645,610,576,547,532,523,...,369,418,484,590,754,969,1201,1460,1716,2002,2319,2690,3055,3368,3666,3901,4082,4258,4372,4392,4231,4256,4419,4495,4501,4472,4486,4499,4453,4404,4443,4472,4428,4353,4306,4284,4318,4311,4321,0
3,706,520,560,572,425,243,271,272,258,276,299,313,320,336,299,274,322,361,358,377,387,396,415,486,564,624,668,680,696,728,753,721,700,713,690,642,592,560,542,547,...,384,418,481,568,693,865,1057,1259,1478,1714,1971,2261,2548,2818,3135,3377,3564,3731,3820,3814,3710,3770,3904,3976,3967,3973,4032,4078,4040,3992,3972,4006,4032,3975,3946,3954,3944,3936,3939,0
4,1120,1027,592,414,407,463,417,365,332,334,356,327,296,325,346,308,262,262,292,312,335,394,430,450,491,570,611,616,631,659,686,705,691,667,624,584,570,549,530,517,...,349,365,398,505,677,877,1116,1410,1717,2015,2339,2670,3028,3415,3713,3928,4131,4296,4445,4508,4305,4232,4422,4554,4550,4524,4532,4578,4584,4555,4502,4485,4479,4445,4364,4290,4268,4235,4272,0


## Droping the rows if ground truth value is zero

In [121]:
print("Percentage of column which will be droped",(data.size - data[data.iloc[:, -1] == 0].size)/data.size,"%")

Percentage of column which will be droped 0.1890561992751774 %


In [0]:
data = data[data.iloc[:, -1] != 0]

# Spliting the data into feature and target

In [0]:
X = data.iloc[:, :-1]
y = data.iloc[:,-1]

# Feature Selection

In [0]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif

X = SelectKBest(f_classif, k=int((image.shape[2]+1)*0.75)).fit_transform(X, y)

# OneHotEncoding in target column

In [0]:
onehotencoder = OneHotEncoder() 
y = onehotencoder.fit_transform(np.array(y).reshape(-1,1)).toarray() 

# Standardizing the feature columna

In [0]:
X = preprocessing.scale(X)

# Spliting the data into training and testing set 

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.60, random_state=1)#0.25 0.15

In [0]:
number = int(X_test.shape[0]/2)

xx_test = X_test[:number, :]
yy_test = y_test[:number, :]
xx_val = X_test[number:, :]
yy_val = y_test[number:, :]

In [0]:
from keras import backend as K
feature = X.shape[1]

if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, feature)
    xx_test = xx_test.reshape(xx_test.shape[0], 1, feature)
    xx_val = xx_val.reshape(xx_val.shape[0], 1, feature)
    input_shape = (1, feature)
else:
    X_train = X_train.reshape(X_train.shape[0], feature, 1)
    xx_test = xx_test.reshape(xx_test.shape[0], feature, 1)
    xx_val = xx_val.reshape(xx_val.shape[0], feature, 1)
    input_shape = (feature, 1)

# Building CNN Model


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D, BatchNormalization

In [131]:
model = Sequential()
model.add(Conv1D(8, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(Conv1D(8, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(16, 3, activation='relu'))
model.add(Conv1D(16, 3, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(32, 3, activation='relu'))
model.add(Conv1D(32, 3, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, 3, activation='relu'))
model.add(Conv1D(64, 3, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(16, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(9, activation='softmax'))
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_41 (Conv1D)           (None, 75, 8)             32        
_________________________________________________________________
conv1d_42 (Conv1D)           (None, 73, 8)             200       
_________________________________________________________________
batch_normalization_52 (Batc (None, 73, 8)             32        
_________________________________________________________________
max_pooling1d_21 (MaxPooling (None, 36, 8)             0         
_________________________________________________________________
conv1d_43 (Conv1D)           (None, 34, 16)            400       
_________________________________________________________________
conv1d_44 (Conv1D)           (None, 32, 16)            784       
_________________________________________________________________
batch_normalization_53 (Batc (None, 32, 16)           

In [132]:
import keras
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=128,
          epochs=100,
          verbose=1,
          validation_data=(xx_val, yy_val))

Train on 59260 samples, validate on 44446 samples
Epoch 1/100
59260/59260 [==============================] - 18s 311us/step - loss: 0.8699 - accuracy: 0.7321 - val_loss: 0.8816 - val_accuracy: 0.8102
Epoch 2/100
59260/59260 [==============================] - 16s 275us/step - loss: 0.3830 - accuracy: 0.8738 - val_loss: 0.1428 - val_accuracy: 0.9612
Epoch 3/100
59260/59260 [==============================] - 16s 273us/step - loss: 0.2725 - accuracy: 0.9120 - val_loss: 0.1316 - val_accuracy: 0.9467
Epoch 4/100
59260/59260 [==============================] - 16s 272us/step - loss: 0.2286 - accuracy: 0.9251 - val_loss: 0.0945 - val_accuracy: 0.9674
Epoch 5/100
59260/59260 [==============================] - 16s 273us/step - loss: 0.2010 - accuracy: 0.9339 - val_loss: 0.0798 - val_accuracy: 0.9707
Epoch 6/100
59260/59260 [==============================] - 16s 278us/step - loss: 0.1853 - accuracy: 0.9386 - val_loss: 0.0748 - val_accuracy: 0.9709
Epoch 7/100
59260/59260 [=========================

In [133]:
score = model.evaluate(xx_test, yy_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

44446/44446 [==============================] - 4s 100us/step
Test loss: 0.0861251611462413
Test accuracy: 0.9818206429481506
